In [56]:
import numpy as np
import torch
import pandas as pd
import matplotlib.pyplot as plt
import os

import tensorflow as tf
import keras as keras
from keras.datasets.mnist import load_data
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout

from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [57]:
dataset = pd.read_csv("TrainData_x1.csv", sep=';')
dataset

,Fluid_lvl_tank102,Pressure_tank103,Pump_speed,Flowmeter,Tank102_set,Pressure_set,Flow_set,Fluid_lvl_tank101_IO_link,Fluid_lvl_tank102_IO_link,Fluid_lvl_MAX_tank101,...,V104,V106,V107,V108,V109,V112,Cycle_nuber,EStop,Work,State
0,80.09615,147.431880,0.000000,0.00,0.0,150.0,0.0,105,80,0,...,0,0,0,1,0,0,1,0,0,2
1,79.75961,150.492770,70.675030,0.31,0.0,150.0,0.0,105,80,0,...,0,0,0,1,0,0,1,0,1,2
2,79.75961,150.747830,70.748890,0.00,0.0,150.0,0.0,105,80,0,...,0,0,0,1,0,0,1,0,1,2
3,79.59135,150.747830,70.525444,0.00,0.0,150.0,0.0,105,80,0,...,0,0,0,1,0,0,1,0,1,2
4,79.75961,149.217390,70.285675,0.00,0.0,150.0,0.0,105,80,0,...,0,0,0,1,0,0,1,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18551,81.27404,7.397102,0.000000,-0.36,0.0,0.0,0.0,105,85,0,...,0,0,1,0,0,1,4,0,1,9
18552,81.10577,7.397102,0.000000,-0.34,0.0,0.0,0.0,105,85,0,...,0,0,1,0,0,1,4,0,1,9
18553,80.60096,7.142029,0.000000,-0.34,0.0,0.0,0.0,105,85,0,...,0,0,1,0,0,1,4,0,1,9
18554,80.43269,7.397102,0.000000,-0.36,0.0,0.0,0.0,105,85,0,...,0,0,1,0,0,1,4,0,1,9


In [58]:
label = pd.read_csv("TrainData_Y1.csv",sep=';')
label

,Failure_F1,Failure_F2,Failure_F3,Failure_F4,Failure_F5
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
18551,0,0,0,0,1
18552,0,0,0,0,1
18553,0,0,0,0,1
18554,0,0,0,0,1


In [59]:
testdataset = pd.read_csv("TestData_x1.csv",sep=';')
testdataset

,Fluid_lvl_tank102,Pressure_tank103,Pump_speed,Flowmeter,Tank102_set,Pressure_set,Flow_set,Fluid_lvl_tank101_IO_link,Fluid_lvl_tank102_IO_link,Fluid_lvl_MAX_tank101,...,V104,V106,V107,V108,V109,V112,Cycle_nuber,EStop,Work,State
0,79.75961,36.475365,0.000000,-0.35,0.0,150.0,0.0,105,85,0,...,0,0,0,0,0,1,6,0,1,2
1,79.59135,208.394210,80.351440,0.27,0.0,150.0,0.0,105,85,0,...,0,0,0,1,0,0,6,0,1,2
2,79.59135,192.834800,75.014730,0.00,0.0,150.0,0.0,105,85,0,...,0,0,0,1,0,0,6,0,1,2
3,79.59135,165.542040,72.681660,0.00,0.0,150.0,0.0,105,85,0,...,0,0,0,1,0,0,6,0,1,2
4,79.59135,158.400010,71.367195,0.00,0.0,150.0,0.0,105,85,0,...,0,0,0,1,0,0,6,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083,81.27404,7.397102,0.000000,-0.34,0.0,0.0,0.0,105,85,0,...,0,0,1,0,0,1,5,0,1,9
2084,81.10577,7.142029,0.000000,-0.36,0.0,0.0,0.0,105,85,0,...,0,0,1,0,0,1,5,0,1,9
2085,80.76923,7.142029,0.000000,-0.35,0.0,0.0,0.0,105,85,0,...,0,0,1,0,0,1,5,0,1,9
2086,80.60096,7.397102,0.000000,-0.35,0.0,0.0,0.0,105,85,0,...,0,0,1,0,0,1,5,0,1,9


In [60]:
testlabel = pd.read_csv("TestData_y1.csv",sep=';')
testlabel

,Failure_F1,Failure_F2,Failure_F3,Failure_F4,Failure_F5
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
2083,0,0,0,0,1
2084,0,0,0,0,1
2085,0,0,0,0,1
2086,0,0,0,0,1


In [61]:
x_train = dataset
y_train = label
x_test = testdataset
y_test = testlabel

In [62]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)


In [63]:
#x = np.asarray(x_train).astype(np.float32)
#y = np.asarray(y_train).astype(np.float32)


In [64]:
model = Sequential()

In [65]:
model.add(Dense(units=128, activation='relu',input_shape=(24,)))

In [66]:
model.add(Dropout(0.2))

In [67]:
#model.add(Dense(units=32, activation='selu'))# sigmoid

In [68]:
#model.add(Dense(units=20, activation='relu'))

In [69]:
model.add(Dense(units=10, activation='gelu'))

In [70]:
model.add(Dense(units=5, activation='softplus')) #softmax

In [71]:
model.compile(
optimizer= keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07), 
loss= keras.losses.CategoricalCrossentropy(
    from_logits=False,
    #label_smoothing=0.0,
    #axis=-1,
),
metrics= ['accuracy'])

In [72]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 128)               3200      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 10)                1290      
                                                                 
 dense_8 (Dense)             (None, 5)                 55        
                                                                 
Total params: 4,545
Trainable params: 4,545
Non-trainable params: 0
_________________________________________________________________


In [73]:
history = model.fit(x_train, y_train, batch_size=255, epochs=100)

Epoch 1/100
73/73 [==============================] - 1s 2ms/step - loss: 0.9062 - accuracy: 0.4117
Epoch 2/100
73/73 [==============================] - 0s 3ms/step - loss: 0.5637 - accuracy: 0.4855
Epoch 3/100
73/73 [==============================] - 0s 3ms/step - loss: 0.4948 - accuracy: 0.5017
Epoch 4/100
73/73 [==============================] - 0s 3ms/step - loss: 0.4591 - accuracy: 0.5134
Epoch 5/100
73/73 [==============================] - 0s 2ms/step - loss: 0.4338 - accuracy: 0.5259
Epoch 6/100
73/73 [==============================] - 0s 2ms/step - loss: 0.4171 - accuracy: 0.5331
Epoch 7/100
73/73 [==============================] - 0s 2ms/step - loss: 0.4026 - accuracy: 0.5410
Epoch 8/100
73/73 [==============================] - 0s 2ms/step - loss: 0.3896 - accuracy: 0.5467
Epoch 9/100
73/73 [==============================] - 0s 3ms/step - loss: 0.3834 - accuracy: 0.5502
Epoch 10/100
73/73 [==============================] - 0s 3ms/step - loss: 0.3735 - accuracy: 0.5528
Epoch 11/

In [74]:
model.evaluate(x_test, y_test, verbose=2)


66/66 - 0s - loss: 1.6022 - accuracy: 0.2069 - 215ms/epoch - 3ms/step


[1.6021645069122314, 0.2068965584039688]

In [75]:
metrics = pd.DataFrame(history.history)
metrics

,loss,accuracy
0,0.906239,0.411727
1,0.563684,0.485503
2,0.494799,0.501724
3,0.459147,0.513365
4,0.433803,0.525922
...,...,...
95,0.316399,0.573292
96,0.315121,0.579220
97,0.315705,0.575878
98,0.315481,0.578842


In [76]:
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(y=metrics['loss'], name='loss'), row=1, col=1)
fig.add_trace(go.Scatter(y=metrics['accuracy'], name='accuracy'), row=2, col=1)
fig.update_layout(width=800)

In [77]:
x_pred=model.predict(x_test)
print(y_test)
print(x_pred)

66/66 [==============================] - 0s 1ms/step
      Failure_F1  Failure_F2  Failure_F3  Failure_F4  Failure_F5
0              0           0           0           0           0
1              0           0           0           0           0
2              0           0           0           0           0
3              0           0           0           0           0
4              0           0           0           0           0
...          ...         ...         ...         ...         ...
2083           0           0           0           0           1
2084           0           0           0           0           1
2085           0           0           0           0           1
2086           0           0           0           0           1
2087           0           0           0           0           1

[2088 rows x 5 columns]
[[7.7223898e-09 2.8739891e-10 6.1157234e-06 2.7297970e-02 1.9937287e-04]
 [1.7168167e-08 3.4182463e-08 3.1097289e-10 8.2672061e-03 3.1327754e-

In [78]:

y_pred=np.argmax(x_pred,axis=1)
y_pred


array([3, 4, 4, ..., 3, 3, 3], dtype=int64)

In [79]:
pred= pd.concat([pd.DataFrame(y_test, columns=['Failure_F1','Failure_F2','Failure_F3','Failure_F4','Failure_F5']), pd.DataFrame(y_pred, columns=['y_pred'])], axis=1)
pred.head(10)

,Failure_F1,Failure_F2,Failure_F3,Failure_F4,Failure_F5,y_pred
0,0,0,0,0,0,3
1,0,0,0,0,0,4
2,0,0,0,0,0,4
3,0,0,0,0,0,4
4,0,0,0,0,0,4
5,0,0,0,0,0,4
6,0,0,0,0,0,4
7,0,0,0,0,0,4
8,0,0,0,0,0,4
9,0,0,0,0,0,4


In [80]:
misclassified = pred[pred['y_test'] != pred['y_pred']]
misclassified.index[:10]

KeyError: 'y_test'